# for_mnm_matching_algorithm



### 설문조사 결과(.xlsx) 불러오기

In [13]:
import pandas as pd

filename = "하우스 메이트 설문결과.xlsx"
df = pd.read_excel(filename)

print(df)

             TIMESTAMP 1. 성별  ...    13. 정리정돈을 신경쓰는 정도                14. 지인 출입
0  2021-10-20 11:29:52    여자  ...                    2            누구든 데려오면 안된다.
1  2021-10-22 01:16:34    남자  ...  내 공간만 아니면 신경쓰이지 않는다  미리 말만 하면 지인이 출입해도 상관없다.
2  2021-10-22 01:29:18    여자  ...    공유하는 공간의 경우 신경쓰인다  미리 말만 하면 지인이 출입해도 상관없다.
3  2021-10-22 01:31:02    남자  ...    공유하는 공간의 경우 신경쓰인다  미리 말만 하면 지인이 출입해도 상관없다.
4  2021-10-22 01:33:35    남자  ...    공유하는 공간의 경우 신경쓰인다  미리 말만 하면 지인이 출입해도 상관없다.
..                 ...   ...  ...                  ...                      ...
67 2021-10-23 00:31:12    남자  ...    공유하는 공간의 경우 신경쓰인다  미리 말만 하면 지인이 출입해도 상관없다.
68 2021-10-23 14:23:31    여자  ...    공유하는 공간의 경우 신경쓰인다  미리 말만 하면 지인이 출입해도 상관없다.
69 2021-10-23 15:38:02    여자  ...    공유하는 공간의 경우 신경쓰인다  미리 말만 하면 지인이 출입해도 상관없다.
70 2021-10-23 16:10:43    여자  ...    공유하는 공간의 경우 신경쓰인다  미리 말만 하면 지인이 출입해도 상관없다.
71 2021-10-23 19:32:47    여자  ...    공유하는 공간의 경우 신경쓰인다  미리 말만 하면 지인이 출입해도 상관없다.

[72 rows x 20 columns]


In [14]:
df.head()

,TIMESTAMP,1. 성별,2. 나이,3. mbti,4. 흡연 유무,4-1. 원하는 메이트의 흡연여부,5. 반려동물과 같이 살 예정이다.,5-1. 반려동물을 키울 경우 어떤 동물인가?,5-2. 메이트의 반려동물 중 허용가능한 것은?,6. 난방/냉방 관련 문제,"7. 벌레를 잡을 수 있는가? (바퀴벌레, 돈벌레, 지네 정도)",7-1. 원하는 메이트,8.집에서 밥을 먹을 경우의 식사 스타일,8-1. 메이트가 집에서 요리하는 게 불편한가요? (냄새 문제 등),9. 선호하는 식사유형,10. 메이트와 물건을 공유해도 괜찮습니까?,11. 방내에서의 소음문제,12. 음주 문제,13. 정리정돈을 신경쓰는 정도,14. 지인 출입
0,2021-10-20 11:29:52,여자,24 ~ 26,infp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,절대 못잡는다,NaN,벌레를 잡을 수 있어야만 한다,직접 요리,NaN,그렇다,NaN,NaN,소음은 없었으면 좋겠다.,2,누구든 데려오면 안된다.
1,2021-10-22 01:16:34,남자,27 ~ 29,intp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,더위를 많이 타서 에어컨을 일찍부터 튼다\t여름에 항상 에어컨을 켜놓는다\t과한 난...,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트가 술주정만 하지 않으면 된다,내 공간만 아니면 신경쓰이지 않는다,미리 말만 하면 지인이 출입해도 상관없다.
2,2021-10-22 01:29:18,여자,20 ~ 23,esfp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지\t조류,더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다\...,절대 못잡는다,벌레를 잡을 수 있어야만 한다,"외부 음식(포장, 배달)",NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,10분 이상의 동영상/음악 소리는 신경쓰인다\t지속적인 알람소리는 신경쓰인다,메이트가 술주정만 하지 않으면 된다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
3,2021-10-22 01:31:02,남자,20 ~ 23,모른다,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,NaN,잡는데 거리낌이 없다,벌레를 못잡아도 상관없다,직접 요리,NaN,같이 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트의 술 문제는 신경쓰지 않는다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
4,2021-10-22 01:33:35,남자,20 ~ 23,intp,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,과한 난방은 답답해하는 편이다,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,괜찮지 않다(안된다),지속적인 알람소리는 신경쓰인다,메이트가 술 냄새를 풍기는 것은 싫다\t메이트가 술을 많이 마신 것 같은 모습을 보...,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.


In [15]:
df.tail()

,TIMESTAMP,1. 성별,2. 나이,3. mbti,4. 흡연 유무,4-1. 원하는 메이트의 흡연여부,5. 반려동물과 같이 살 예정이다.,5-1. 반려동물을 키울 경우 어떤 동물인가?,5-2. 메이트의 반려동물 중 허용가능한 것은?,6. 난방/냉방 관련 문제,"7. 벌레를 잡을 수 있는가? (바퀴벌레, 돈벌레, 지네 정도)",7-1. 원하는 메이트,8.집에서 밥을 먹을 경우의 식사 스타일,8-1. 메이트가 집에서 요리하는 게 불편한가요? (냄새 문제 등),9. 선호하는 식사유형,10. 메이트와 물건을 공유해도 괜찮습니까?,11. 방내에서의 소음문제,12. 음주 문제,13. 정리정돈을 신경쓰는 정도,14. 지인 출입
67,2021-10-23 00:31:12,남자,24 ~ 26,isfj,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,여름에 항상 에어컨을 켜놓는다,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,"외부 음식(포장, 배달)",신경쓰지 않는다(불편하지 않다),같이 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트가 술주정만 하지 않으면 된다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
68,2021-10-23 14:23:31,여자,20 ~ 23,isfj,비흡연자,비흡연자,아니다,NaN,강아지,추위를 많이 타서 난방을 일찍부터 튼다,잡는데 거리낌이 없다,벌레를 못잡아도 상관없다,"외부 음식(포장, 배달)",신경쓰지 않는다(불편하지 않다),같이 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,10분 이상의 말소리(통화/게임)은 신경쓰인다\t10분 이상의 동영상/음악 소리는 ...,메이트가 술주정만 하지 않으면 된다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
69,2021-10-23 15:38:02,여자,30대,모른다,비흡연자,상관없음,아니다,NaN,어떤 동물이든 괜찮다.,해당하는 상황 없음,잡는데 거리낌이 없다,벌레를 못잡아도 상관없다,직접 요리,신경쓰지 않는다(불편하지 않다),같이 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,지속적인 알람소리는 신경쓰인다,메이트의 술 문제는 신경쓰지 않는다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
70,2021-10-23 16:10:43,여자,20 ~ 23,intp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,더위를 많이 타서 에어컨을 일찍부터 튼다\t여름에 항상 에어컨을 켜놓는다\t과한 난...,절대 못잡는다,벌레를 못잡아도 상관없다,"외부 음식(포장, 배달)",신경쓰지 않는다(불편하지 않다),혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,지속적인 알람소리는 신경쓰인다\t10분 이전의 말소리(통화/게임)도 신경쓰인다\t1...,메이트의 술 문제는 신경쓰지 않는다\t메이트가 술주정만 하지 않으면 된다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
71,2021-10-23 19:32:47,여자,24 ~ 26,istp,비흡연자,비흡연자,아니다,NaN,강아지,해당하는 상황 없음,절대 못잡는다,벌레를 잡을 수 있어야만 한다,직접 요리,신경쓰지 않는다(불편하지 않다),혼자 먹는 걸 선호한다,괜찮지 않다(안된다),10분 이상의 말소리(통화/게임)은 신경쓰인다\t10분 이상의 동영상/음악 소리는 ...,메이트가 술을 많이 마신 것 같은 모습을 보이면 싫다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   TIMESTAMP                              72 non-null     datetime64[ns]
 1   1. 성별                                  72 non-null     object        
 2   2. 나이                                  72 non-null     object        
 3   3. mbti                                72 non-null     object        
 4   4. 흡연 유무                               72 non-null     object        
 5   4-1. 원하는 메이트의 흡연여부                     72 non-null     object        
 6   5. 반려동물과 같이 살 예정이다.                    72 non-null     object        
 7   5-1. 반려동물을 키울 경우 어떤 동물인가?              15 non-null     object        
 8   5-2. 메이트의 반려동물 중 허용가능한 것은?             72 non-null     object        
 9   6. 난방/냉방 관련 문제                         71 non-null     object      

### 결측치 처리
샘플이 충분하지 않은 경우 pandas의 fillna() 함수 사용

In [17]:
df.isnull().sum()

TIMESTAMP                                 0
1. 성별                                     0
2. 나이                                     0
3. mbti                                   0
4. 흡연 유무                                  0
4-1. 원하는 메이트의 흡연여부                        0
5. 반려동물과 같이 살 예정이다.                       0
5-1. 반려동물을 키울 경우 어떤 동물인가?                57
5-2. 메이트의 반려동물 중 허용가능한 것은?                0
6. 난방/냉방 관련 문제                            1
7. 벌레를 잡을 수 있는가? (바퀴벌레, 돈벌레, 지네 정도)       1
7-1. 원하는 메이트                              0
8.집에서 밥을 먹을 경우의 식사 스타일                    0
8-1. 메이트가 집에서 요리하는 게 불편한가요? (냄새 문제 등)     8
9. 선호하는 식사유형                              0
10. 메이트와 물건을 공유해도 괜찮습니까?                  1
11. 방내에서의 소음문제                            2
12. 음주 문제                                 1
13. 정리정돈을 신경쓰는 정도                         0
14. 지인 출입                                 0
dtype: int64

In [18]:
# DataFrame의 칼럼 이름 확인
df.columns

Index(['TIMESTAMP', '1. 성별', '2. 나이', '3. mbti', '4. 흡연 유무',
       '4-1. 원하는 메이트의 흡연여부', '5. 반려동물과 같이 살 예정이다.',
       '5-1. 반려동물을 키울 경우 어떤 동물인가?', '5-2. 메이트의 반려동물 중 허용가능한 것은?',
       '6. 난방/냉방 관련 문제', '7. 벌레를 잡을 수 있는가? (바퀴벌레, 돈벌레, 지네 정도)', '7-1. 원하는 메이트',
       '8.집에서 밥을 먹을 경우의 식사 스타일', '8-1. 메이트가 집에서 요리하는 게 불편한가요? (냄새 문제 등)',
       '9. 선호하는 식사유형', '10. 메이트와 물건을 공유해도 괜찮습니까?', '11. 방내에서의 소음문제',
       '12. 음주 문제', '13. 정리정돈을 신경쓰는 정도', '14. 지인 출입'],
      dtype='object')

In [19]:
df.columns[5]

'4-1. 원하는 메이트의 흡연여부'

In [20]:
df['3. mbti']

0     infp
1     intp
2     esfp
3      모른다
4     intp
      ... 
67    isfj
68    isfj
69     모른다
70    intp
71    istp
Name: 3. mbti, Length: 72, dtype: object

In [21]:
# 인덱스 이름 생성
df.index.name = 'udi'
df.head()

,TIMESTAMP,1. 성별,2. 나이,3. mbti,4. 흡연 유무,4-1. 원하는 메이트의 흡연여부,5. 반려동물과 같이 살 예정이다.,5-1. 반려동물을 키울 경우 어떤 동물인가?,5-2. 메이트의 반려동물 중 허용가능한 것은?,6. 난방/냉방 관련 문제,"7. 벌레를 잡을 수 있는가? (바퀴벌레, 돈벌레, 지네 정도)",7-1. 원하는 메이트,8.집에서 밥을 먹을 경우의 식사 스타일,8-1. 메이트가 집에서 요리하는 게 불편한가요? (냄새 문제 등),9. 선호하는 식사유형,10. 메이트와 물건을 공유해도 괜찮습니까?,11. 방내에서의 소음문제,12. 음주 문제,13. 정리정돈을 신경쓰는 정도,14. 지인 출입
udi,,,,,,,,,,,,,,,,,,,,
0,2021-10-20 11:29:52,여자,24 ~ 26,infp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,절대 못잡는다,NaN,벌레를 잡을 수 있어야만 한다,직접 요리,NaN,그렇다,NaN,NaN,소음은 없었으면 좋겠다.,2,누구든 데려오면 안된다.
1,2021-10-22 01:16:34,남자,27 ~ 29,intp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지,더위를 많이 타서 에어컨을 일찍부터 튼다\t여름에 항상 에어컨을 켜놓는다\t과한 난...,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트가 술주정만 하지 않으면 된다,내 공간만 아니면 신경쓰이지 않는다,미리 말만 하면 지인이 출입해도 상관없다.
2,2021-10-22 01:29:18,여자,20 ~ 23,esfp,비흡연자,비흡연자,아니다,NaN,고양이\t강아지\t조류,더위를 많이 타서 에어컨을 일찍부터 튼다\t추위를 많이 타서 난방을 일찍부터 튼다\...,절대 못잡는다,벌레를 잡을 수 있어야만 한다,"외부 음식(포장, 배달)",NaN,혼자 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,10분 이상의 동영상/음악 소리는 신경쓰인다\t지속적인 알람소리는 신경쓰인다,메이트가 술주정만 하지 않으면 된다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
3,2021-10-22 01:31:02,남자,20 ~ 23,모른다,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,NaN,잡는데 거리낌이 없다,벌레를 못잡아도 상관없다,직접 요리,NaN,같이 먹는 걸 선호한다,사용하기 전 허락을 받으면 괜찮다,신경쓰지 않는다.,메이트의 술 문제는 신경쓰지 않는다,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.
4,2021-10-22 01:33:35,남자,20 ~ 23,intp,비흡연자,비흡연자,아니다,NaN,어떠한 동물도 안된다.,과한 난방은 답답해하는 편이다,시키면 잡을 수 있다,벌레를 못잡아도 상관없다,직접 요리,NaN,혼자 먹는 걸 선호한다,괜찮지 않다(안된다),지속적인 알람소리는 신경쓰인다,메이트가 술 냄새를 풍기는 것은 싫다\t메이트가 술을 많이 마신 것 같은 모습을 보...,공유하는 공간의 경우 신경쓰인다,미리 말만 하면 지인이 출입해도 상관없다.


In [22]:
# 인덱스는 컬럼에 들어가지 않는다.
df.columns

Index(['TIMESTAMP', '1. 성별', '2. 나이', '3. mbti', '4. 흡연 유무',
       '4-1. 원하는 메이트의 흡연여부', '5. 반려동물과 같이 살 예정이다.',
       '5-1. 반려동물을 키울 경우 어떤 동물인가?', '5-2. 메이트의 반려동물 중 허용가능한 것은?',
       '6. 난방/냉방 관련 문제', '7. 벌레를 잡을 수 있는가? (바퀴벌레, 돈벌레, 지네 정도)', '7-1. 원하는 메이트',
       '8.집에서 밥을 먹을 경우의 식사 스타일', '8-1. 메이트가 집에서 요리하는 게 불편한가요? (냄새 문제 등)',
       '9. 선호하는 식사유형', '10. 메이트와 물건을 공유해도 괜찮습니까?', '11. 방내에서의 소음문제',
       '12. 음주 문제', '13. 정리정돈을 신경쓰는 정도', '14. 지인 출입'],
      dtype='object')

### 컬럼 이름 변경

In [23]:
df = df.rename(columns={'1. 성별': 'sex', '2. 나이': 'age', '3. mbti': 'mbti', '4. 흡연 유무': 'user_smoking', '4-1. 원하는 메이트의 흡연여부': 'mate_smoking'})
df.columns

Index(['TIMESTAMP', 'sex', 'age', 'mbti', 'user_smoking', 'mate_smoking',
       '5. 반려동물과 같이 살 예정이다.', '5-1. 반려동물을 키울 경우 어떤 동물인가?',
       '5-2. 메이트의 반려동물 중 허용가능한 것은?', '6. 난방/냉방 관련 문제',
       '7. 벌레를 잡을 수 있는가? (바퀴벌레, 돈벌레, 지네 정도)', '7-1. 원하는 메이트',
       '8.집에서 밥을 먹을 경우의 식사 스타일', '8-1. 메이트가 집에서 요리하는 게 불편한가요? (냄새 문제 등)',
       '9. 선호하는 식사유형', '10. 메이트와 물건을 공유해도 괜찮습니까?', '11. 방내에서의 소음문제',
       '12. 음주 문제', '13. 정리정돈을 신경쓰는 정도', '14. 지인 출입'],
      dtype='object')

In [24]:
df = df.rename(columns={'5. 반려동물과 같이 살 예정이다.': 'user_pet'})